In [25]:
import numpy as np
import arepo
import h5py as h5
import matplotlib.pyplot as plt
import matplotlib as mpl
from numba import njit

In [4]:
basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas/'

In [30]:
snap = 100
sn = arepo.Snapshot(basepath + 'runs/MW3_MHG0.25_GSE2/lvl4/output', 
                             snap, combineFiles=True)

fname = basepath + 'anlys/MC/MW3_MHG0.25_GSE2-lvl4/MC_Prop_'+str(snap).zfill(3)+'.h5'
MC = h5.File(fname, mode='r')

In [14]:
MC['PartType5']['TracerID'][:10]

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype=uint32)

In [15]:
sn.part5.ParentID[:10]

array([ 696729, 5919775, 7141183,  486712, 5595554, 6295524, 7133376,
        522773,  801725, 6382953], dtype=uint32)

In [17]:
key_sort = np.argsort(sn.part5.ParentID)

In [31]:
MC['PartType5']['ParentID']

<HDF5 dataset "ParentID": shape (6616488,), type "<u4">

In [29]:
MC['PartType5'].keys()

<KeysViewHDF5 ['Coordinates', 'GFM_Metallicity', 'Membership', 'PartType', 'RotatedCoordinates', 'RotatedVelocities', 'TracerID', 'Velocities']>

In [34]:
def get_massMC(ParticleIDs, ParentIDs, TracerMass):
    massMC = np.zeros(np.shape(ParticleIDs))

    partid_sort = np.argsort(ParticleIDs)
    parentid_sort = np.argsort(ParentIDs)

    i = j = 0
    while i < len(ParentIDs):
        if j == len(ParticleIDs):
            print(i, j)
        
        parentid = ParentIDs[parentid_sort[i]]
        partid = ParticleIDs[partid_sort[j]]

        if parentid == partid:
            massMC[partid_sort[j]] += TracerMass
            i += 1
        else:
            j += 1
    
    return massMC

In [41]:
is_gas = MC['PartType5/PartType'][:] == 0
is_GSE = MC['PartType5/Membership'][:] == 2
is_GSE_gas = np.logical_and(is_gas, is_GSE)

In [44]:
massMC = get_massMC(sn.part0.ParticleIDs, MC['PartType5']['ParentID'][is_GSE_gas], 
                    MC['Header'].attrs['TracerMass'])

In [48]:
massMC[massMC>0]

array([6.75183117e-06, 1.05028485e-05, 9.00244156e-06, ...,
       1.20032554e-05, 1.27534589e-05, 3.00081385e-06])

In [49]:
MC['Header'].attrs['TracerMass']

7.502034637444493e-07